<h1 style=font:garmond;font-size:50px;color:orange;text-align:left>Predict AI</h1>

In [ ]:
import csv
import numpy as np
import pandas as pd

In [ ]:
column_names = ['A', 'B', 'C', 'D', 'E', 'F']
df = pd.read_csv('data.csv', names=column_names)

print(df)

<h2 style=font:garmond;font-size:36px;color:orange;text-align:left>Transformation</h2>

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_dataframe = pd.DataFrame(data=transformed_dataset, index=df.index)

<h2 style=font:garmond;font-size:36px;color:orange;text-align:left>Modeling</h2>

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from keras.optimizers import SGD

In [ ]:
number_of_rows = df.values.shape[0]
window_length = 25
number_of_features = df.values.shape[1]

In [ ]:
train = np.empty([number_of_rows-window_length, window_length, number_of_features], dtype=float)
label = np.empty([number_of_rows-window_length, number_of_features], dtype=float)

for i in range(0, number_of_rows-window_length):
    train[i] = transformed_dataframe.iloc[i:i+window_length, 0: number_of_features]
    label[i] = transformed_dataframe.iloc[i+window_length: i+window_length+1, 0: number_of_features]

print(f'{train.shape}\n{label.shape}')

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(42, activation='relu', input_shape=(window_length, number_of_features), return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(42, activation='relu', return_sequences=False)))
model.add(Dropout(0.2))
model.add(Dense(number_of_features))

In [ ]:
learning_rate = 0.1
momentum = 0.8
decay_rate = 0.005

sgd = SGD(learning_rate=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

In [ ]:
model.compile(loss='mse', optimizer=sgd, metrics='accuracy')

In [ ]:
batch_size = 32
epochs = 100

history = model.fit(train, label, batch_size=batch_size, epochs=epochs)

In [ ]:
model.summary()

<h2 style=font:garmond;font-size:36px;color:orange;text-align:left>Result</h2>

In [ ]:
to_predict = np.array(df.iloc[-window_length:])
scaled_to_predict = scaler.transform(to_predict)

scaled_prediction = model.predict(np.array([scaled_to_predict]))
np_prediction = scaler.inverse_transform(scaled_prediction).astype(int)[0]

print(*np_prediction, sep=', ')

In [ ]:
with open('storage.csv', 'a', encoding='UTF-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(np_prediction)

<h2 style=font:garmond;font-size:36px;color:orange;text-align:left>Post-training Analysis</h2>

In [29]:
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
column_names = ['A', 'B', 'C', 'D', 'E', 'F']
df2 = pd.read_csv('storage.csv', names=column_names)

print(df2)

In [ ]:
today = datetime.today()
now = datetime.now()

date_formatted = today.strftime('%m/%d/%y')
time_formatted = now.strftime('%H:%M')

title: str = f'{date_formatted}-{time_formatted}'

plt.figure(figsize=(19.5,9.5))
plt.plot(history.history['accuracy'])
plt.title(title)
plt.ylabel('accuracy', fontsize=14)
plt.xlabel('epoch', fontsize=14)
plt.grid()
plt.show()